# Setup libraries

In [1]:
import os
import numpy as np
from astropy.table import Table

# Cleaning

This function follows the cuts in Table 4 of Mandelbaum 2018.

In [2]:
def make_cuts(catalog_in):
    # Basic flag cuts
    select = catalog_in["idetect_is_primary"] == True
    select &= catalog_in["ideblend_skipped"] == False
    select &= catalog_in["iflags_badcentroid"] == False
    select &= catalog_in["icentroid_sdss_flags"] == False
    select &= catalog_in["iflags_pixel_edge"] == False
    select &= catalog_in["iflags_pixel_interpolated_center"] == False
    select &= catalog_in["iflags_pixel_saturated_center"] == False
    select &= catalog_in["iflags_pixel_cr_center"] == False
    select &= catalog_in["iflags_pixel_bad"] == False
    select &= catalog_in["iflags_pixel_suspect_center"] == False
    select &= catalog_in["iflags_pixel_clipped_any"] == False
    select &= catalog_in["ishape_hsm_regauss_flags"] == False
    select &= catalog_in["ishape_hsm_regauss_sigma"] != np.nan
    select &= catalog_in["icmodel_flux_flags"] == False
    select &= catalog_in["iclassification_extendedness"] != 0

    # Cuts on object properties
    select &= catalog_in["icmodel_flux"] / catalog_in["icmodel_flux_err"] >= 10.0
    select &= catalog_in["ishape_hsm_regauss_resolution"] >= 0.3
    select &= (
        catalog_in["ishape_hsm_regauss_e1"] ** 2
        + catalog_in["ishape_hsm_regauss_e2"] ** 2
        < 4.0
    )
    select &= (catalog_in["ishape_hsm_regauss_sigma"] <= 0.4) & (
        catalog_in["ishape_hsm_regauss_sigma"] >= 0
    )
    select &= catalog_in["icmodel_mag"] - catalog_in["a_i"] <= 24.5
    select &= catalog_in["iblendedness_abs_flux"] < (10 ** (-0.375))

    # Additional flux requirements - at least 2 of 4 must be satisfied
    flux_g = catalog_in["gcmodel_flux"] / catalog_in["gcmodel_flux_err"] >= 5.0
    flux_r = catalog_in["rcmodel_flux"] / catalog_in["rcmodel_flux_err"] >= 5.0
    flux_z = catalog_in["zcmodel_flux"] / catalog_in["zcmodel_flux_err"] >= 5.0
    flux_y = catalog_in["ycmodel_flux"] / catalog_in["ycmodel_flux_err"] >= 5.0

    flux_count = (
        flux_g.astype(int)
        + flux_r.astype(int)
        + flux_z.astype(int)
        + flux_y.astype(int)
    )
    select &= flux_count >= 2

    catalog_out = catalog_in[select]

    return catalog_out

In [3]:
for root, dirs, files in os.walk("clusters"):
    for file in files:
        if file.endswith("_raw_shear_catalog.fits"):
            raw_catalog = Table.read(os.path.join(root, file))

            catalog = make_cuts(raw_catalog)

            catalog.write(
                os.path.join(
                    root, file.replace("_raw_shear_catalog", "_shear_catalog")
                ),
                overwrite=True,
            )

/tmp/ipykernel_929667/4292754624.py:20: RuntimeWarning: invalid value encountered in divide
  select &= catalog_in["icmodel_flux"] / catalog_in["icmodel_flux_err"] >= 10.0
